# Project 2


The purpose of this project is to build a model, that will be able to distinguish between different seal calls. 

Grey Seals are known for their wide repertoire of communications which range from clapping [[1]](https://www.smithsonianmag.com/smart-news/first-scientists-film-grey-seals-clapping-show-their-strength-180974133/) to vocalisations so complex that they have been shown to imitate the sounds of vowels and other building blocks of human speech [[2]](https://www.pbs.org/wgbh/nova/article/seals-mimic-speech/).

This project will look at specific vocalisations called Rupes and Moans.

There are 3 types of rupes A, B and C are distinguished from each other based on frequency, duration and modulation.

The dataset used in this stuy comes froma  study into the vocalisations of grey seals off of malin head. [[3]](https://www.mdpi.com/2077-1312/12/1/118)

In [1]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import numpy as np
import pandas as pd
from matplotlib.colors import LogNorm
import glob
import os

c:\Users\kolea\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
#File index
file =  'data/Samples Grey Seal/Rupes A and B/5713.210809120002'  #from PPT at time 892-896 (Rupe B)
#file = 'Guttural rupe\\5711.211013040024'
#file = 'Rupes A and B\\5713.210825190002'
#file =          'Moan\\5713.210902110002'  #from PPT at time 212 seconds

#Read the 2 files
sample_rate, samples = wavfile.read(file+'.wav')
annot_file_path = file +'.Table.1.selections.txt'

#Read the file into a DataFrame
df = pd.read_csv(file +'.Table.1.selections.txt', sep='\t')

#Display the first few rows of the DataFrame
print(df.head())

In [16]:
# combine all rupe call annotation into one dataframe

folder = "data/Samples Grey Seal/Rupes A and B"

combined_df = pd.DataFrame()

for file in os.listdir(folder):
  if file.endswith('.txt'): # only want the annotated files
    file_path = os.path.join(folder, file)

    # add to dataframe
    df = pd.read_csv(file_path, sep='\t')

    combined_df = pd.concat([combined_df, df], ignore_index=True)

combined_df.head(20)

#save to CSV
combined_df.to_csv('data/combined_rupes_data.csv', index=False)

## Calculate the Spectrogram

The signal.spectrogram function from scipy [[4]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html) will produce a visual represenatation of the audio file showing how the frequencies change over time. 

In [40]:
frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate, nperseg=2456, nfft=4096, noverlap=1228, window='hann')       #Try nfft=8192 if computer permits

In [ ]:
frequencies

In [18]:
spectrogram[spectrogram < 0.001] = 0.001    #Trim off all the tiny values so that log scale displays correctly

#Trim the frequency. All seal sounds are under 1000 Hz
fmin = 20 # Hz
fmax = 1000 # Hz
freq_slice = np.where((frequencies >= fmin) & (frequencies <= fmax))

#keep only frequencies of interest
frequencies = frequencies[freq_slice]
spectrogram = spectrogram[freq_slice,:][0]

In [19]:
def overlay_annotations(ax, df, annotation_colors):
    #Track labels to ensure they are added only once in the legend
    added_labels = set()

    for _, row in df.iterrows():
        start_time = row['Begin Time (s)']
        end_time = row['End Time (s)']
        low_freq = row['Low Freq (Hz)']
        high_freq = row['High Freq (Hz)']
        annotation = row['Annotation']

        #Skip if the annotation is not in the defined colors
        if annotation not in annotation_colors:
            continue

        #Draw rectangles
        ax.add_patch(
            plt.Rectangle(
                (start_time, low_freq),  #Bottom Left corner
                end_time - start_time,  #Width (time)
                high_freq - low_freq,  #Height (frequency)
                edgecolor=annotation_colors[annotation],
                facecolor='none',
                linewidth=2,
                label=annotation if annotation not in added_labels else None  #Add label once
            )
        )
        added_labels.add(annotation)  #Mark label as added

    ax.legend(loc='upper right')     #Add legend

In [21]:
def update_colormap(event):
    #Get current view limits
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    #Find indices corresponding to the current view limits
    x_indices = np.where((times >= xlim[0]) & (times <= xlim[1]))[0]
    y_indices = np.where((frequencies >= ylim[0]) & (frequencies <= ylim[1]))[0]

    #Handle cases where no data is visible
    if len(x_indices) == 0 or len(y_indices) == 0:
        return

    #Extract the visible data
    data_visible = spectrogram[np.ix_(y_indices, x_indices)]
    #data_visible = np.log(spectrogram)[np.ix_(y_indices, x_indices)]

    #Compute new color limits
    vmin = np.nanmin(data_visible)
    vmax = np.nanmax(data_visible)

    #Update the color limits of the pcolormesh
    pc.set_clim(vmin=vmin, vmax=vmax)
    
    #Update the colorbar to reflect the new color limits
    cbar.update_normal(pc)

    #Redraw the figure
    plt.draw()
    

In [ ]:
#Define colors for annotations
annotation_colors = {
    "Rupe A": "red",
    "Rupe B": "green",
    "Growl B": "yellow",
    "Rupe C" : "purple",
    "Moan": "pink",
    "G rupe" : "blue"
}

pc = plt.pcolormesh(times, frequencies, spectrogram, norm=LogNorm(), cmap='Spectral_r')
#pc = plt.pcolormesh(times, frequencies, np.log(spectrogram))
cbar = plt.colorbar(pc)
#plt.imshow(spectrogram)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')

#Get the current axes
ax = plt.gca()
overlay_annotations(ax, df, annotation_colors)

#Connect the update function to the axes limit change events
ax.callbacks.connect('xlim_changed', update_colormap)
ax.callbacks.connect('ylim_changed', update_colormap)

plt.show()

In [ ]:
#Define time and frequency limits
time_start, time_end = 891, 897  # Time range in seconds
freq_start, freq_end = 20, 600  # Frequency range in Hz

#Find indices for the time range
time_indices = np.where((times >= time_start) & (times <= time_end))[0]

#Find indices for the frequency range
freq_indices = np.where((frequencies >= freq_start) & (frequencies <= freq_end))[0]

#Extract the portion of the spectrogram
spectrogram_sub = spectrogram[freq_indices][:, time_indices]
frequencies_sub = frequencies[freq_indices]
times_sub = times[time_indices]
print(spectrogram_sub.shape)

#Plot the original and sub-portion spectrograms
plt.figure(figsize=(10, 6))

plt.subplot(2, 1, 1)
plt.pcolormesh(times, frequencies, spectrogram, norm=LogNorm(), cmap='Spectral_r')
plt.title('Original Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.colorbar(label='Power [dB]')

plt.subplot(2, 1, 2)
plt.pcolormesh(times_sub, frequencies_sub, spectrogram_sub, norm=LogNorm(), cmap='Spectral_r')
plt.title('Extracted Portion of Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.colorbar(label='Power [dB]')

plt.tight_layout()
plt.show()

In [ ]:
#Define time and frequency limits
time_start, time_end = 892.5, 893.2  # Time range in seconds
freq_start, freq_end = 20, 600  # Frequency range in Hz

#Find indices for the time range
time_indices = np.where((times >= time_start) & (times <= time_end))[0]

#Find indices for the frequency range
freq_indices = np.where((frequencies >= freq_start) & (frequencies <= freq_end))[0]

#Extract the portion of the spectrogram
spectrogram_sub = spectrogram[freq_indices][:, time_indices]
frequencies_sub = frequencies[freq_indices]
times_sub = times[time_indices]
print("Spectrogram size: ", spectrogram_sub.shape)

#Plot the original and sub-portion spectrograms
plt.figure(figsize=(10, 6))

plt.subplot(2, 1, 1)
plt.pcolormesh(times, frequencies, spectrogram, norm=LogNorm(), cmap='Spectral_r')
plt.title('Original Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.colorbar(label='Power [dB]')

plt.subplot(2, 1, 2)
plt.pcolormesh(times_sub, frequencies_sub, spectrogram_sub, norm=LogNorm(), cmap='Spectral_r')
plt.title('Extracted Portion of Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.colorbar(label='Power [dB]')

plt.tight_layout()
plt.show()

In [ ]:
plt.savefig('spectrogram.png')
#plt.clf()
print(spectrogram_sub)

In [1]:
#Looking at Spectrogram

In [2]:
# Folder containing the text files
folder_path = "data/Samples Grey Seal/Rupes A and B/"

# Initialize an empty list to store DataFrames
dataframes = []

# Get all .txt files in the folder
txt_files = glob.glob(os.path.join(folder_path, "*.txt"))

# Iterate through each text file
for txt_file in txt_files:
    # Get the base name of the corresponding .wav file
    base_name = os.path.splitext(os.path.basename(txt_file))[0].replace(".Table.1.selections", "")
    wav_file = f"{base_name}.wav"  # Name of the associated .wav file

    # Load the text file into a DataFrame
    df = pd.read_csv(txt_file, sep='\t', header='infer')

    # Add a column for the corresponding .wav file
    df['Audio File'] = wav_file

    # Append the DataFrame to the list
    dataframes.append(df)

# Combine all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
combined_df.head()

# Save the combined DataFrame to a CSV file for reference
combined_df.to_csv(os.path.join("data/combined_calls.csv"), index=False)

,Selection,View,Channel,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Delta Time (s),Delta Freq (Hz),Avg Power Density (dB FS/Hz),Annotation
0,1,Spectrogram 1,1,13.433865,13.703649,53.191,324.361,0.2698,271.170,-79.66,Rupe B
1,2,Spectrogram 1,1,26.072555,26.283886,53.191,376.259,0.2113,323.068,-84.94,Rupe B
2,3,Spectrogram 1,1,53.382769,53.528888,66.489,478.723,0.1461,412.234,-89.26,Rupe A
3,4,Spectrogram 1,1,65.997665,66.110075,59.347,415.430,0.1124,356.083,-86.86,Rupe A
4,5,Spectrogram 1,1,81.003161,81.266117,59.347,376.259,0.2630,316.912,-82.32,Rupe B
5,6,Spectrogram 1,1,86.568316,86.874071,59.347,400.006,0.3058,340.659,-83.07,Rupe B
6,7,Spectrogram 1,1,97.068628,97.333469,59.347,498.471,0.2648,439.124,-85.93,Rupe A
7,8,Spectrogram 1,1,97.899554,98.048987,74.184,356.083,0.1494,281.899,-86.57,Rupe A
8,9,Spectrogram 1,1,105.623018,105.750196,71.281,570.247,0.1272,498.966,-88.29,Rupe A
9,10,Spectrogram 1,1,106.102974,106.167550,57.025,498.966,0.0646,441.941,-86.36,Rupe A


In [3]:
#look at full dataframe for every call
combined_df

In [4]:


# PLot the distribution of call durations

# Plot the distribution of Delta Time (s)
plt.figure(figsize=(10, 6))
plt.hist(combined_df['Delta Time (s)'], bins=20, color='blue', edgecolor='black', alpha=0.7)
plt.title("Distribution of Call Lengths (Delta Time)", fontsize=14)
plt.xlabel("Call Length (Delta Time (s)", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.grid(axis='y', alpha=0.75)
plt.show()


In [8]:
# Check what calls are included and how many of each

unique_calls = combined_df['Annotation'].value_counts(sort=False)
unique_calls

Some of the calls were un-identified or were identified as Trrot or Type 4 A. I am unsure what type of call this is. However they are in low number and will be excluded from our project going forward. 

In [ ]:
#List of Annotation types that we want to keep

annotations_to_include = ['Rupe A', 'Rupe B', 'Rupe C', 'Growl B', 'Guttural rupe']
filtered_df = combined_df[combined_df['Annotation'].isin(annotations_to_include)]

filtered_df['Annotation'].value_counts(sort=False)


In [ ]:
# Iterate through each row in the combined DataFrame
for index, row in combined_df.iterrows():
    # Get the associated audio file
    audio_path = os.path.join(folder_path, row['Audio File'])

    # Load the audio file
    if not os.path.exists(audio_path):
        print(f"Audio file {audio_path} not found. Skipping...")
        continue

    sample_rate, samples = wavfile.read(audio_path)

    # Calculate the midpoint of the call
    call_midpoint = (row['Begin Time (s)'] + row['End Time (s)']) / 2

    # Define the 3-second time window around the call
    time_start = call_midpoint - 1.5
    time_end = call_midpoint + 1.5

    # Convert time to sample indices
    start_sample = int(time_start * sample_rate)
    end_sample = int(time_end * sample_rate)

    # Handle edge cases where the segment exceeds the audio boundaries
    if start_sample < 0:
        start_sample = 0
    if end_sample > len(samples):
        end_sample = len(samples)

    # Extract the audio segment
    segment = samples[start_sample:end_sample]

    # Compute the spectrogram for the segment
    frequencies, times, spectrogram = signal.spectrogram(
        segment, sample_rate, nperseg=1024, nfft=2048, noverlap=512, window='hann'
    )

    # Adjust times to reflect the original audio time range
    times = times + time_start  # Shift the spectrogram time axis to match the original audio timestamps

    # Save the spectrogram data as a .npz file
    output_file = os.path.join(f"data/processed/Spectrograms/{row['Audio File']}_call_{index + 1}_{row['Annotation'].replace(' ', '_')}.npz")
    np.savez_compressed(
        output_file,
        spectrogram=spectrogram,
        frequencies=frequencies,
        times=times,
        annotation=row['Annotation'],
        time_start=time_start,
        time_end=time_end
    )

    print(f"Processed call {index + 1} from {row['Audio File']} and saved as .npz.")

## References

- 1 https://www.smithsonianmag.com/smart-news/first-scientists-film-grey-seals-clapping-show-their-strength-180974133/
- 2 https://www.pbs.org/wgbh/nova/article/seals-mimic-speech/
- 3 https://www.mdpi.com/2077-1312/12/1/118
- 4 https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html
- https://towardsdatascience.com/audio-deep-learning-made-simple-part-1-state-of-the-art-techniques-da1d3dff2504
- https://medium.com/@okezieowen/audio-deep-learning-in-plain-english-b52843deb64e